In [1]:
print('Baseline')

# This is so that you don't have to restart the kernel everytime you edit hmm.py

%load_ext autoreload
%autoreload 2

Baseline


In [4]:
from preprocess import load_arguments, load_label, load_values_from_json, combine_columns, split_arguments
from model import train_bert_model
import os

# Load arguments
df_arguments = load_arguments('data/arguments-training.tsv')

# Load json
human_values = load_values_from_json('data/value-categories.json')

df_labels = load_label("data/labels-training.tsv", human_values["1"])
df_full_level = combine_columns(df_arguments, df_labels)
train_arguments, valid_arguments = split_arguments(df_full_level)

train_arguments.info()
valid_arguments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176 entries, 3401 to 3031
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Argument ID                 4176 non-null   object
 1   Conclusion                  4176 non-null   object
 2   Stance                      4176 non-null   object
 3   Premise                     4176 non-null   object
 4   Achievement                 4176 non-null   int64 
 5   Benevolence: caring         4176 non-null   int64 
 6   Benevolence: dependability  4176 non-null   int64 
 7   Conformity: interpersonal   4176 non-null   int64 
 8   Conformity: rules           4176 non-null   int64 
 9   Face                        4176 non-null   int64 
 10  Hedonism                    4176 non-null   int64 
 11  Humility                    4176 non-null   int64 
 12  Power: dominance            4176 non-null   int64 
 13  Power: resources            4176 non-null   i

## Random Baseline

In [5]:
from pprint import pprint

from baseline import random_prediction
from metrics import compute_metrics, print_metrics
from utils import extract_true_labels



random_prediction_values = random_prediction(human_values['1'], train_arguments)

metrics = compute_metrics((random_prediction_values, 
                           extract_true_labels(train_arguments, human_values['1'])), human_values['1'])

pprint(metrics)

Random Prediction
Added predictions for 4176 arguments
{'accuracy_thresh': 0.27624520659446716,
 'f1-score': {'Achievement': 0.411,
              'Benevolence: caring': 0.421,
              'Benevolence: dependability': 0.257,
              'Conformity: interpersonal': 0.08,
              'Conformity: rules': 0.343,
              'Face': 0.134,
              'Hedonism': 0.081,
              'Humility': 0.151,
              'Power: dominance': 0.157,
              'Power: resources': 0.18,
              'Security: personal': 0.529,
              'Security: societal': 0.448,
              'Self-direction: action': 0.405,
              'Self-direction: thought': 0.296,
              'Stimulation': 0.112,
              'Tradition': 0.194,
              'Universalism: concern': 0.466,
              'Universalism: nature': 0.13,
              'Universalism: objectivity': 0.293,
              'Universalism: tolerance': 0.241,
              'avg-f1-score': 0.266},
 'marco-avg-f1score': 0.266}


## All 1 predictions


In [6]:
from baseline import all_ones

one_values = all_ones(human_values['1'], train_arguments)

metrics = compute_metrics((one_values, 
                           extract_true_labels(train_arguments, human_values['1'])), human_values['1'])

pprint(metrics)

Predicting all 1
Added predictions for 4176 arguments
{'accuracy_thresh': 0.17182710766792297,
 'f1-score': {'Achievement': 0.427,
              'Benevolence: caring': 0.452,
              'Benevolence: dependability': 0.258,
              'Conformity: interpersonal': 0.081,
              'Conformity: rules': 0.376,
              'Face': 0.135,
              'Hedonism': 0.08,
              'Humility': 0.153,
              'Power: dominance': 0.164,
              'Power: resources': 0.193,
              'Security: personal': 0.547,
              'Security: societal': 0.477,
              'Self-direction: action': 0.407,
              'Self-direction: thought': 0.304,
              'Stimulation': 0.114,
              'Tradition': 0.2,
              'Universalism: concern': 0.55,
              'Universalism: nature': 0.129,
              'Universalism: objectivity': 0.301,
              'Universalism: tolerance': 0.242,
              'avg-f1-score': 0.28},
 'marco-avg-f1score': 0.28}


In [9]:
bert_model_evaluation = train_bert_model(train_arguments, os.path.join('model/', 'bert-base-uncased'), human_values['1'], valid_arguments)
print(bert_model_evaluation['eval_f1-score'])

loading configuration file config.json from cache at /Users/hemantjoshi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/hemantjoshi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b

  0%|          | 0/4176 [00:00<?, ?ex/s]

  0%|          | 0/1044 [00:00<?, ?ex/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /Users/hemantjoshi/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABE

Step,Training Loss,Validation Loss,Accuracy Thresh,F1-score,Marco-avg-f1score
500,0.409700,0.358689,0.856034,"{'Achievement': 0.57, 'Benevolence: caring': 0.36, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.0, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.25, 'Security: personal': 0.66, 'Security: societal': 0.62, 'Self-direction: action': 0.37, 'Self-direction: thought': 0.54, 'Stimulation': 0.0, 'Tradition': 0.01, 'Universalism: concern': 0.56, 'Universalism: nature': 0.0, 'Universalism: objectivity': 0.0, 'Universalism: tolerance': 0.0, 'avg-f1-score': 0.2}",0.200000
1000,0.342900,0.330594,0.867337,"{'Achievement': 0.61, 'Benevolence: caring': 0.39, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.35, 'Face': 0.0, 'Hedonism': 0.0, 'Humility': 0.0, 'Power: dominance': 0.0, 'Power: resources': 0.44, 'Security: personal': 0.7, 'Security: societal': 0.68, 'Self-direction: action': 0.48, 'Self-direction: thought': 0.56, 'Stimulation': 0.0, 'Tradition': 0.36, 'Universalism: concern': 0.65, 'Universalism: nature': 0.68, 'Universalism: objectivity': 0.05, 'Universalism: tolerance': 0.14, 'avg-f1-score': 0.3}",0.300000
1500,0.298100,0.323357,0.869875,"{'Achievement': 0.62, 'Benevolence: caring': 0.43, 'Benevolence: dependability': 0.0, 'Conformity: interpersonal': 0.0, 'Conformity: rules': 0.35, 'Face': 0.0, 'Hedonism': 0.07, 'Humility': 0.0, 'Power: dominance': 0.04, 'Power: resources': 0.41, 'Security: personal': 0.69, 'Security: societal': 0.69, 'Self-direction: action': 0.58, 'Self-direction: thought': 0.58, 'Stimulation': 0.07, 'Tradition': 0.31, 'Universalism: concern': 0.71, 'Universalism: nature': 0.78, 'Universalism: objectivity': 0.28, 'Universalism: tolerance': 0.17, 'avg-f1-score': 0.34}",0.340000
2000,0.265100,0.322529,0.871791,"{'Achievement': 0.62, 'Benevolence: caring': 0.44, 'Benevolence: dependability': 0.01, 'Conformity: interpersonal': 0.09, 'Conformity: rules': 0.47, 'Face': 0.0, 'Hedonism': 0.19, 'Humility': 0.0, 'Power: dominance': 0.14, 'Power: resources': 0.38, 'Security: personal': 0.66, 'Security: societal': 0.69, 'Self-direction: action': 0.61, 'Self-direction: thought': 0.62, 'Stimulation': 0.07, 'Tradition': 0.47, 'Universalism: concern': 0.7, 'Universalism: nature': 0.79, 'Universalism: objectivity': 0.27, 'Universalism: tolerance': 0.18, 'avg-f1-score': 0.37}",0.370000
2500,0.229800,0.328061,0.870833,"{'Achievement': 0.62, 'Benevolence: caring': 0.46, 'Benevolence: dependability': 0.04, 'Conformity: interpersonal': 0.05, 'Conformity: rules': 0.46, 'Face': 0.0, 'Hedonism': 0.37, 'Humility': 0.04, 'Power: dominance': 0.08, 'Power: resources': 0.37, 'Security: personal': 0.73, 'Security: societal': 0.68, 'Self-direction: action': 0.61, 'Self-direction: thought': 0.64, 'Stimulation': 0.17, 'Tradition': 0.46, 'Universalism: concern': 0.68, 'Universalism: nature': 0.77, 'Universalism: objectivity': 0.33, 'Universalism: tolerance': 0.32, 'avg-f1-score': 0.39}",0.390000
3000,0.201500,0.335690,0.868726,"{'Achievement': 0.61, 'Benevolence: caring': 0.44, 'Benevolence: dependability': 0.09, 'Conformity: interpersonal': 0.09, 'Conformity: rules': 0.45, 'Face': 0.0, 'Hedonism': 0.33, 'Humility': 0.15, 'Power: dominance': 0.24, 'Power: resources': 0.44, 'Security: personal': 0.68, 'Security: societal': 0.67, 'Self-direction: action': 0.62, 'Self-direction: thought': 0.66, 'Stimulation': 0.17, 'Tradition': 0.43, 'Universalism: concern': 0.71, 'Universalism: nature': 0.74, 'Universalism: objectivity': 0.34, 'Universalism: tolerance': 0.3, 'avg-f1-score': 0.41}",0.410000
3500,0.175400,0.345241,0.864799,"{'Achievement': 0.64, 'Benevolence: caring': 0.49, 'Benevolence: dependability': 0.18, 'Conformity: interpersonal': 0.14, 'Conformity: rules': 0.46, 'Face': 0.0, 'Hedonism': 0.43, 'Humility': 0.16, 'Power: dominance': 0.22, 'Power: resources': 0.49, 'Security: personal': 0.67, 'Securi

***** Running Evaluation *****
  Num examples = 1044
  Batch size = 8
Saving model checkpoint to model/bert-base-uncased/checkpoint-500
Configuration saved in model/bert-base-uncased/checkpoint-500/config.json
Model weights saved in model/bert-base-uncased/checkpoint-500/pytorch_model.bin
tokenizer config file saved in model/bert-base-uncased/checkpoint-500/tokenizer_config.json
Special tokens file saved in model/bert-base-uncased/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 8
Saving model checkpoint to model/bert-base-uncased/checkpoint-1000
Configuration saved in model/bert-base-uncased/checkpoint-1000/config.json
Model weights saved in model/bert-base-uncased/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in model/bert-base-uncased/checkpoint-1000/tokenizer_config.json
Special tokens file saved in model/bert-base-uncased/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples =

KeyboardInterrupt: 